# Analysis of drug-to-drug information sharing

This notebook is designed to create the heatmap figure from a set of partially drug-blind experiments. We run 1600 total models to have a training and test set for the elastic net models for each drug.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.stats import pearsonr
import os
from sklearn.linear_model import  ElasticNetCV, ElasticNet
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import r2_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from seaborn import clustermap
import colorcet as cc
import seaborn as sns
import matplotlib.patches as mpatches

In [ ]:
def build_drug_oneHot(input_file, index_map, len_uniques):
    curr_df = pd.read_csv(input_file, sep='\t', header=None)
    curr_uniques = list(set(curr_df.iloc[:,1]))
    temp = np.zeros(len_uniques)
    idx = [index_map[x] for x in curr_uniques]
    temp[idx] = 1
    return temp

def calculate_corr_perDrug(file, index_map):
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        f = open(file, 'r')
        lines = f.readlines()
        lines = lines[1:]

        drugs = [x.split('\t')[1] for x in lines]
        cancerTypes = [x.split('\t')[2] for x in lines]
        pred_responses = [x.split('\t')[3] for x in lines]
        true_responses = [x.split('\t')[4].strip() for x in lines]
        # I accidentally saved the tensor value in my experiments so we have to parse it, please don't judge me too harshly. Still works, thanks Python typecasting!
        # TODO would be to change that in results saving
        pred_responses_float = [float(s.split('[', 1)[1].split(']')[0]) for s in pred_responses]
        true_responses_float = [float(s.split('[', 1)[1].split(']')[0]) for s in true_responses] 

        unique_drugs = list(set(drugs))
        temp = np.zeros(len(unique_drugs))
        for drug in unique_drugs:
            curr_pred = [pred_responses_float[i] for i,x in enumerate(drugs) if x == drug]
            curr_true = [true_responses_float[i] for i,x in enumerate(drugs) if x == drug]
            if len(set(curr_pred)) == 1:
                print(file)
            curr_corr = pearsonr(curr_pred, curr_true)
            
            temp[index_map[drug]] = curr_corr.statistic
            
        if w:
            print(file)
        return temp
    
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    fig, ax = plt.subplots(figsize=(60,15), dpi=600)
    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)
    print(linkage_matrix.shape)
    # Plot the corresponding dendrogram
    plt.figure(figsize=(60,15), dpi=600)
    dendrogram(linkage_matrix, ax=ax, **kwargs)
    drug_names = [unique_map_reversed[int(x.get_text())] if '(' not in x.get_text() else x.get_text() for x in ax.get_xticklabels()]
    print(drug_names)
    ax.set_yticks([])
    ax.set_xticklabels(drug_names)

def calculate_linkages(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    return linkage_matrix

In [ ]:
# Get index map of all unique drugs from original input file
train_df = pd.read_csv(f'input-files/gdsc_ec50.txt', sep='\t', header=None)
unique_drugs = list(dict.fromkeys(train_df.iloc[:,1]))
unique_map = {x:i for i,x in enumerate(unique_drugs)}
len_unique = len(unique_drugs)

The below assumes that:

(1) You saved all experiments with a constant prefix.

(2) The seed is the last element in the file name.

(3) All experiments are in the same directory for training set composition and test results.

In [ ]:
exp_prefix = 'your_experiment_here'
train_dir = 'train_dir_here'
test_dir = 'test_dir_here'

train_oneHots_dict = {int(f.strip('.txt').split('_')[-1]):build_drug_oneHot(os.path.join(train_dir, f), unique_map, len_unique) 
                for f in os.listdir(train_dir) 
                if f.startswith(exp_prefix)}

test_performance_byDrug = {int(f.strip('.txt').split('_')[-1]):calculate_corr_perDrug(os.path.join(test_dir,f), unique_map)
                           for f in os.listdir(test_dir)
                           if f.startswith(exp_prefix)}

In [ ]:
# Line up results by seed as the key
total = [(train_oneHots_dict[key],test_performance_byDrug[key]) for key in test_performance_byDrug.keys()]
X,Y = zip(*total)

In [ ]:
# Calculate l1_ratio and alpha by 10-fold cross validation and grid search for a selected amount of drugs
num_drugs_to_sample = 10
drug_idx_to_test = np.random.choice(len_unique, size=num_drugs_to_sample)
for d in drug_idx_to_test:
    ss = ShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
    ratios = np.arange(0, 1, 0.01)
    alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
    model = ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=ss)
    curr_Y = [y[int(d)] for y in Y]
    model.fit(X,curr_Y)
    print('alpha: %f' % model.alpha_)
    print('l1_ratio_: %f' % model.l1_ratio_)

In [ ]:
# Set your alpha and l1_ratio according to above results

my_alpha = 0.01
my_l1 = 0.10

In [ ]:
# Plot average accuracy for each drug (sorted) across all train splits
unique_map_reversed = {v:k for k,v in unique_map.items()}
accuracies = np.array([[y[i] for y in Y] for i in range(len_unique)]) # Get list of accuracies for each drug from list of seeded runs
avg_accuracies = [sum(x)/len(x) for x in accuracies]
avg_accuracies_dict = {unique_map_reversed[i]:sum(x)/len(x) for i,x in enumerate(accuracies)}
avg_accuracies.sort(reverse=True)
x=np.array(list(range(1, len(avg_accuracies)+1)))
y=np.array(avg_accuracies)
plt.figure(dpi=300)
plt.bar(x, y, color='blue')
plt.xlabel('drug')
plt.ylabel('avg accuracy')

In [ ]:
'''
Get all coefs for ElasticNet model
    Features: Drugs present in training set across all models [Shape: (n_models, n_drugs)]
    Targets: Accuracies for i'th drug across all models [Shape: (n_models)]

Uses 10-fold shuffle split
'''
ss = ShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
coefs = []
drug_avg_scores = {}
for j in range(len(accuracies)): # Loop over all drugs
    curr_coefs = np.zeros(len_unique)
    curr_Y = accuracies[j]
    scores = []
    for train_idx, test_idx in ss.split(X):
        model = ElasticNet(alpha=my_alpha, l1_ratio=my_l1, max_iter=1000)
        X_train = [X[i] for i in train_idx]
        X_test = [X[i] for i in test_idx]
        Y_train = [curr_Y[i] for i in train_idx]
        Y_test = [curr_Y[i] for i in test_idx]
        model.fit(X_train, Y_train)
        y_pred = model.predict(X_test)
        scores.append(r2_score(Y_test,y_pred))
        curr_coefs += model.coef_
        # plt.scatter(y_pred, Y_test, c='blue')
    coefs.append(curr_coefs)
    # print(f'Average for {unique_map_reversed[j]} is {sum(scores)/len(scores)}')
    drug_avg_scores[unique_map_reversed[j]] = sum(scores)/len(scores)

In [ ]:
clust_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0, metric='euclidean', linkage='ward', compute_distances=True)
clust_model.fit(coefs)
linkages = calculate_linkages(clust_model)

In [ ]:
gdsc_moa_df = pd.read_csv('screened_compounds_rel_8.5.csv')
print(f'There are {len(set(gdsc_moa_df.iloc[:,5]))} unique pathways in GDSC')
drug_pathways_dict = {y:x for y,x in zip(gdsc_moa_df.loc[:, 'DRUG_NAME'],gdsc_moa_df.loc[:,'TARGET_PATHWAY'])}
print(drug_pathways_dict)

In [ ]:
# Depending on the dataset you're using, you may have to define certain drugs. Will throw error if they're not in pathways dict
drug_pathways = pd.Series([drug_pathways_dict[drug] for drug in unique_map.keys()])
palette = sns.color_palette(cc.glasbey, n_colors=25)
palette = palette.as_hex()
lut = dict(zip(drug_pathways.unique(), palette))
row_colors = drug_pathways.map(lut)

In [ ]:
abs_coefs = np.abs(coefs)
g = clustermap(coefs, col_linkage=linkages, row_linkage=linkages, cbar_pos = (0.05,0.85,0.15,.01), cbar_kws={"orientation": "horizontal"}, vmin=-1, vmax=1, cmap='icefire',dendrogram_ratio=(.2, .2),figsize=(20,20), xticklabels=1, rasterized=True, col_colors=row_colors.values)#
drug_names = [unique_map_reversed[int(x.get_text())] if '(' not in x.get_text() else x.get_text() for x in g.ax_heatmap.get_xticklabels()] # Dendrogram is originally labeled with index of drug
g.ax_heatmap.set_xticklabels(drug_names, verticalalignment='top')
g.ax_heatmap.tick_params(right=False, labelright=False, bottom=False, labelbottom=False, labeltop=True, labelsize=5)
g.ax_heatmap.tick_params(axis='x', pad=70)
g.set_dpi = 300
pos = g.ax_row_dendrogram.get_position()
print(pos)
pos.x1 = .23
g.ax_row_dendrogram.set_position(pos)
pos = g.ax_col_dendrogram.get_position()
print(pos)
pos.y1=0.95
pos.y0=0.868
g.ax_col_dendrogram.set_position(pos)
cbar = g.ax_cbar
cbar.tick_params(labelsize=15)
plt.setp(g.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)
markers = [mpatches.Patch(color=color, label=label) for label,color in lut.items()]
plt.legend(handles=markers, bbox_to_anchor=(6.05,4), loc='upper left', fontsize=15, title='Pathway')
pos = g.ax_col_colors.get_position()
pos.y0=0.83
pos.y1=0.865
g.ax_col_colors.set_position(pos)
# plt.savefig('dendro_heatmap.svg', format='svg', dpi=600,  bbox_inches='tight')